TD : Nettoyage des Données Textuelles en Python pour le NLP

1 Introduction au nettoyage des données

Questions
1. Pourquoi est-il important de nettoyer les données textuelles pour le NLP ?
Améliorer la qualité des données, réduire la complexité du texte, augmenter la performance des modèles et éliminer les biais.

2. Quels sont les principaux types de bruit dans les textes collectés ?
Le bruit dans les textes collectés peut provenir de diverses sources : Fautes d’orthographe et de grammaire, caractères spéciaux et ponctuation, mots inutiles, langage informel et abréviations, langage mixte et emojis, données redondantes et bruit provenant des métadonnées


2 Nettoyage des textes : suppression des caractères inutiles

In [25]:
import requests

url = ('https://newsapi.org/v2/everything?'
       'q=Spider Man&'
       'from=2024'
       'sortBy=popularity&'
       'apiKey=af9b1c16ddfa4c17a8c38de16d7dfbc3')

response = requests.get(url)

print(response.json())

{'status': 'ok', 'totalResults': 3247, 'articles': [{'source': {'id': None, 'name': 'Gizmodo.com'}, 'author': 'Germain Lussier', 'title': 'The Larger Sony Spider-Man Universe May End This Week', 'description': 'Kraven the Hunter might be the last Spider-Man film without Spider-Man for some time.', 'url': 'https://gizmodo.com/sony-spiderman-universe-dead-kraven-the-hunter-tom-holland-2000536639', 'urlToImage': 'https://gizmodo.com/app/uploads/2024/12/spiderman-across-spiderverse-poster.jpg', 'publishedAt': '2024-12-10T17:50:13Z', 'content': 'Morbin’ time may finally be over. No more researching spiders in the Amazon. Venom has been stung. And Kraven is, apparently, no longer what Sony is cravin’. This week marks the release of Kraven the… [+2194 chars]'}, {'source': {'id': None, 'name': 'Gizmodo.com'}, 'author': 'James Whitbrook', 'title': 'The Comic Book History Behind Kraven‘s Wildest Spider-Man Connection', 'description': "We may never know where Sony could've gone in still trying to

In [26]:
response.json()

{'status': 'ok',
 'totalResults': 3247,
 'articles': [{'source': {'id': None, 'name': 'Gizmodo.com'},
   'author': 'Germain Lussier',
   'title': 'The Larger Sony Spider-Man Universe May End This Week',
   'description': 'Kraven the Hunter might be the last Spider-Man film without Spider-Man for some time.',
   'url': 'https://gizmodo.com/sony-spiderman-universe-dead-kraven-the-hunter-tom-holland-2000536639',
   'urlToImage': 'https://gizmodo.com/app/uploads/2024/12/spiderman-across-spiderverse-poster.jpg',
   'publishedAt': '2024-12-10T17:50:13Z',
   'content': 'Morbin’ time may finally be over. No more researching spiders in the Amazon. Venom has been stung. And Kraven is, apparently, no longer what Sony is cravin’. This week marks the release of Kraven the… [+2194 chars]'},
  {'source': {'id': None, 'name': 'Gizmodo.com'},
   'author': 'James Whitbrook',
   'title': 'The Comic Book History Behind Kraven‘s Wildest Spider-Man Connection',
   'description': "We may never know where Son

Installation de nltk pour procéder au nltk pour procéder au nettoyage des données

In [11]:
pip install nltk


  Using cached nltk-3.9.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
Using cached nltk-3.9.1-py3-none-any.whl (1.5 MB)
Using cached click-8.1.7-py3-none-any.whl (97 kB)


In [22]:
import pandas as pd
import numpy as np
import nltk
import re

Conversion des réponses obtenues en dataframe pour bien visualiser les informations avant nettoyage.

In [27]:
df = pd.DataFrame(response.json())
df.head()

,status,totalResults,articles
0,ok,3247,"{'source': {'id': None, 'name': 'Gizmodo.com'}..."
1,ok,3247,"{'source': {'id': None, 'name': 'Gizmodo.com'}..."
2,ok,3247,"{'source': {'id': None, 'name': 'Gizmodo.com'}..."
3,ok,3247,"{'source': {'id': None, 'name': 'Gizmodo.com'}..."
4,ok,3247,"{'source': {'id': None, 'name': '[Removed]'}, ..."


Création de listes avec les informations leur correspondant (dans chaque colonne) pour les enregistrer dans un fichier csv.

In [33]:
import requests
import pandas as pd

# Requête pour obtenir les articles
url = ('https://newsapi.org/v2/everything?'
       'q=Spider Man&'
       'from=2024-01-01&'
       'sortBy=popularity&'
       'apiKey=af9b1c16ddfa4c17a8c38de16d7dfbc3')

response = requests.get(url)
data = response.json()

# Liste pour stocker les articles
articles = data.get('articles', [])

# Créer des listes pour chaque colonne
authors = []
titles = []
descriptions = []
urls = []
published_dates = []

# Remplir les listes avec les informations des articles
for article in articles:
    authors.append(article.get('author', 'N/A'))
    titles.append(article.get('title', 'N/A'))
    descriptions.append(article.get('description', 'N/A'))
    urls.append(article.get('url', 'N/A'))
    published_dates.append(article.get('publishedAt', 'N/A'))

# Créer un DataFrame avec les colonnes appropriées
df = pd.DataFrame({
    'Author': authors,
    'Title': titles,
    'Description': descriptions,
    'URL': urls,
    'Published Date': published_dates
})

# Nom du fichier CSV
output_file = 'Articles_NLP.csv'

# Sauvegarder les résultats dans un fichier CSV
df.to_csv(output_file, index=False, encoding="utf-8")

# Message de confirmation
print(f"\nLes articles ont été sauvegardés dans le fichier {output_file}.")



Les articles ont été sauvegardés dans le fichier Articles_NLP.csv.


In [2]:
pip install requests pandas nltk

Note: you may need to restart the kernel to use updated packages.


In [11]:
import requests
import pandas as pd
import re
from datetime import datetime, timedelta
import time

class NewsAPI:
    def __init__(self, api_key):
        """Initialisation avec la clé API"""
        self.api_key = api_key
        self.base_url = 'https://newsapi.org/v2'
        self.headers = {
            'X-Api-Key': api_key,
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        }

    def search_articles(self, keyword, from_date=None):
        """Recherche d'articles avec gestion des erreurs"""
        endpoint = f"{self.base_url}/everything"
        
        # Paramètres de la requête API
        params = {
            'q': keyword,
            'language': 'en',
            'sortBy': 'publishedAt',
            'pageSize': 100
        }
        
        if from_date:
            params['from'] = from_date

        try:
            response = requests.get(
                endpoint, 
                headers=self.headers,
                params=params
            )
            
            # Pause pour respecter les limites de l'API
            time.sleep(1)
            
            # Vérification du statut HTTP de la réponse
            if response.status_code != 200:
                print(f"Erreur HTTP {response.status_code}")
                print(f"Message: {response.text}")
                return []
            
            data = response.json()
            
            # Vérification du statut de la réponse API
            if data.get('status') != 'ok':
                print(f"Erreur API: {data.get('message', 'Erreur inconnue')}")
                return []
                
            return data.get('articles', [])
            
        except requests.exceptions.RequestException as e:
            print(f"Erreur de connexion: {e}")
            return []
        except Exception as e:
            print(f"Erreur inattendue: {e}")
            return []

class TextCleaner:
    @staticmethod
    def clean(text):
        """Nettoyage basique du texte"""
        if not isinstance(text, str):
            return ""
        
        # Conversion en minuscules
        text = text.lower()
        
        # Suppression des URLs
        text = re.sub(r'http\S+|www\S+', '', text)
        
        # Suppression des occurrences de 'removed'
        text = re.sub(r'removed', '', text)
        
        # Suppression des caractères spéciaux et des chiffres
        text = re.sub(r'[^\w\s]', ' ', text)
        
        # Normalisation des espaces
        text = re.sub(r'\s+', ' ', text).strip()
        
        return text
    
    @staticmethod
    def extract_words(text, min_length=3):
        """Extraction des mots avec longueur minimale"""
        cleaned = TextCleaner.clean(text)
        return [word for word in cleaned.split() if len(word) >= min_length]

def process_articles(articles):
    """Traitement des articles et création du DataFrame"""
    processed_data = []
    
    for article in articles:
        # Extraction et nettoyage du texte
        title = article.get('title', '')
        description = article.get('description', '')
        content = article.get('content', '')
        
        # Texte combiné pour l'analyse
        full_text = f"{title} {description} {content}"
        cleaned_text = TextCleaner.clean(full_text)
        words = TextCleaner.extract_words(full_text)
        
        # Création de l'entrée
        entry = {
            'titre': title,
            'description': description,
            'texte_nettoye': cleaned_text,
            'mots': ' '.join(words),
            'nombre_mots': len(words),
            'url': article.get('url', ''),
            'auteur': article.get('author', ''),
            'source': article.get('source', {}).get('name', ''),
            'date_publication': article.get('publishedAt', '')
        }
        
        processed_data.append(entry)
    
    return pd.DataFrame(processed_data)

def main():
    # Configuration de l'API et des paramètres de recherche
    API_KEY = 'af9b1c16ddfa4c17a8c38de16d7dfbc3'
    KEYWORD = 'Spider Man'
    
    # Calculer la date de début (30 jours en arrière par défaut)
    FROM_DATE = (datetime.now() - timedelta(days=30)).strftime('%Y-%m-%d')
    
    OUTPUT_FILE = 'articles_spiderman.csv'
    
    print(f"Démarrage de la recherche pour '{KEYWORD}' à partir de '{FROM_DATE}'...")
    
    # Initialisation de l'API
    news_api = NewsAPI(API_KEY)
    
    # Récupération des articles
    articles = news_api.search_articles(KEYWORD, FROM_DATE)
    
    if not articles:
        print("Aucun article trouvé.")
        return
    
    print(f"Nombre d'articles trouvés : {len(articles)}")
    
    # Afficher quelques exemples d'articles bruts pour inspection
    for i, article in enumerate(articles[:5]):
        print(f"\nArticle {i+1} brut :")
        print(article)
    
    # Traitement des articles
    print("Traitement des articles...")
    df = process_articles(articles)
    
    # Sauvegarde des résultats dans un fichier CSV
    print(f"Sauvegarde dans {OUTPUT_FILE}...")
    df.to_csv(OUTPUT_FILE, index=False, encoding='utf-8')
    
    # Affichage des statistiques
    print("\nRésumé :")
    print(f"- Total articles : {len(df)}")
    if not df.empty:
        print(f"- Moyenne mots/article : {df['nombre_mots'].mean():.0f}")
        print(f"- Sources uniques : {df['source'].nunique()}")
    
    print("\nTraitement terminé !")
    return df

if __name__ "__main__":
    df = main()


Démarrage de la recherche pour 'Spider Man' à partir de '2024-11-19'...
Nombre d'articles trouvés : 100

Article 1 brut :
{'source': {'id': None, 'name': 'Securityaffairs.com'}, 'author': 'Pierluigi Paganini', 'title': 'Irish Data Protection Commission (DPC) fined Meta €251 million for a 2018 data breach', 'description': 'Meta has been fined €251M ($263M) for a 2018 data breach affecting millions in the EU, marking another penalty for violating privacy laws. The Irish Data Protection Commission (DPC) fined Meta €251 million ($263M) for a 2018 data breach impacting 29 million F…', 'url': 'https://securityaffairs.com/172100/laws-and-regulations/dpc-fined-meta-e251-million.html', 'urlToImage': 'https://securityaffairs.com/wp-content/uploads/2022/04/Meta-logo.png', 'publishedAt': '2024-12-18T09:18:19Z', 'content': 'Irish Data Protection Commission (DPC) fined Meta 251 million for a 2018 data breach\r\n\xa0|\xa0Texas Tech University data breach impacted 1.4 million individuals\r\n\xa0|\xa0T

In [12]:
df

,titre,description,texte_nettoye,mots,nombre_mots,url,auteur,source,date_publication
0,Irish Data Protection Commission (DPC) fined M...,Meta has been fined €251M ($263M) for a 2018 d...,irish data protection commission dpc fined met...,irish data protection commission dpc fined met...,79,https://securityaffairs.com/172100/laws-and-re...,Pierluigi Paganini,Securityaffairs.com,2024-12-18T09:18:19Z
1,"Bob Persichetti, Justin K Thompson to direct '...",Bob ​Persichetti was part of the film's direct...,bob persichetti justin k thompson to direct sp...,bob persichetti justin thompson direct spider ...,79,https://economictimes.indiatimes.com/news/inte...,PTI,The Times of India,2024-12-18T08:31:33Z
2,The Mask APT is back after 10 years of silence,Kaspersky researchers linked a new wave of cyb...,the mask apt is back after 10 years of silence...,the mask apt back after years silence kaspersk...,69,https://securityaffairs.com/172093/apt/the-mas...,Pierluigi Paganini,Securityaffairs.com,2024-12-18T08:20:55Z
3,"2024 LEGO Advent Calendars, Day 18 [Feature]","Jingle Bells, something smells... like LEGO! J...",2024 lego advent calendars day 18 feature jing...,2024 lego advent calendars day feature jingle ...,66,https://www.brothers-brick.com/2024/12/18/2024...,Jake Forbes,Brothers-brick.com,2024-12-18T08:00:27Z
4,Tom Hardy's Venom Franchise Box Office Beat Th...,Tom Hardy's Venom has managed to earn more at ...,tom hardy s venom franchise box office beat th...,tom hardy venom franchise box office beat thre...,63,https://screenrant.com/venom-movie-box-office-...,Ollie Bradley,Screen Rant,2024-12-18T03:40:18Z
...,...,...,...,...,...,...,...,...,...
95,Paul Allor & Luca Claretti Launch a New Marvel...,"Marvel Unlimited, the digital streaming servic...",paul allor luca claretti launch a new marvel r...,paul allor luca claretti launch new marvel riv...,79,https://bleedingcool.com/comics/paul-allor-luc...,Rich Johnston,Bleeding Cool News,2024-12-16T17:27:08Z
96,Kraven the Hunter’s Failure Confirms One of Ho...,Kraven the Hunter’s failure at the box office ...,kraven the hunter s failure confirms one of ho...,kraven the hunter failure confirms one hollywo...,61,http://www.vulture.com/article/kraven-the-hunt...,Chris Lee,Vulture,2024-12-16T17:24:27Z
97,Sydney Sweeney Had A Wild Strategy For Keeping...,"Sweeney kept Glen Powell guessing, and the res...",sydney sweeney had a wild strategy for keeping...,sydney sweeney had wild strategy for keeping g...,63,https://www.cinemablend.com/movies/sydney-swee...,Ryan LaBee,CinemaBlend,2024-12-16T17:13:45Z
98,"io9’s Favorite Replays, Re-Reads, and Rewatche...","Here are the TV shows, movies, games, and book...",io9 s favorite replays re reads and rewatches ...,io9 favorite replays reads and rewatches 2024 ...,54,https://gizmodo.com/io9s-favorite-replays-re-r...,Io9 Staff,Gizmodo.com,2024-12-16T17:00:27Z
